In [ ]:
import torch, mmengine, mmcv, mmdet, mmdet3d, spconv
print(torch.__version__)
print(torch.version.cuda)
print(mmengine.__version__)
print(mmcv.__version__)
print(mmdet.__version__)
print(mmdet3d.__version__)
print(spconv.__version__)

In [1]:
import argparse
import logging
import os
import os.path as osp
import torch

from mmengine.config import Config, DictAction
from mmengine.logging import print_log
from mmengine.registry import RUNNERS
from mmengine.runner import Runner

from mmdet3d.utils import replace_ceph_backend

from projects.mmdet3d_plugin.models.detectors import CmtDetector
import time
from mmengine.structures import InstanceData

In [2]:
cfg = Config.fromfile('projects/configs/fusion/my_cmt_kitti.py')
# cfg = Config.fromfile('../mmdetection3d/projects/BEVFusion/configs/my_bevfusion.py')
cfg.work_dir = osp.abspath('./work_dirs')
runner = Runner.from_cfg(cfg)

/media/helloalone/新加卷/ubuntu_anaconda_envs/python38/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/media/helloalone/新加卷/ubuntu_anaconda_envs/python38/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


03/19 13:20:01 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 | packaged by conda-forge | (default, Oct 10 2023, 15:44:36) [GCC 12.3.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 1028313016
    GPU 0: NVIDIA GeForce RTX 3060
    CUDA_HOME: /usr/local/cuda-12.2
    NVCC: Cuda compilation tools, release 12.2, V12.2.91
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 2.1.2
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -genco

/media/helloalone/新加卷/ubuntu_anaconda_envs/python38/lib/python3.8/site-packages/mmdet/models/task_modules/builder.py:17: UserWarning: ``build_sampler`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_sampler`` would be deprecated soon, please use '
/media/helloalone/新加卷/ubuntu_anaconda_envs/python38/lib/python3.8/site-packages/mmdet/models/task_modules/builder.py:39: UserWarning: ``build_assigner`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_assigner`` would be deprecated soon, please use '
/media/helloalone/新加卷/ubuntu_anaconda_envs/python38/lib/python3.8/site-packages/mmdet/models/task_modules/builder.py:46: UserWarning: ``build_sampler`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_sampler`` would be deprecated soon, please use '


03/19 13:20:06 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
03/19 13:20:06 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

In [3]:
runner.model.train()

CmtDetector(
  (data_preprocessor): Det3DDataPreprocessor()
  (pts_voxel_encoder): HardSimpleVFE()
  (pts_middle_encoder): SparseEncoder(
    (conv_input): SparseSequential(
      (0): SubMConv3d(4, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
      (1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (encoder_layers): SparseSequential(
      (encoder_layer1): SparseSequential(
        (0): SparseBasicBlock(
          (conv1): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
          (bn1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (conv2): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], ou

In [4]:
with torch.no_grad():
    data_batch = next(iter(runner.train_dataloader))
    data_batch = runner.model.data_preprocessor(data_batch, training=False)
    batch_inputs_dict = data_batch['inputs']
    batch_data_samples = data_batch['data_samples']
    imgs = batch_inputs_dict.get('imgs', None)
    points = batch_inputs_dict.get('points', None)
    img_metas = [item.metainfo for item in batch_data_samples]

    img_feats = runner.model.extract_img_feat(imgs, img_metas)
    voxels, num_points, coors = runner.model.voxelize(points)
    voxel_features = runner.model.pts_voxel_encoder(voxels, num_points, coors)
    batch_size = coors[-1, 0] + 1
    x1 = runner.model.pts_middle_encoder(voxel_features, coors, batch_size)
    x2 = runner.model.pts_backbone(x1)
    if runner.model.with_pts_neck:
        x3 = runner.model.pts_neck(x2)

03/19 13:20:07 - mmengine - INFO - load 2207 Pedestrian database infos in DataBaseSampler
03/19 13:20:07 - mmengine - INFO - load 14357 Car database infos in DataBaseSampler
03/19 13:20:07 - mmengine - INFO - load 734 Cyclist database infos in DataBaseSampler
03/19 13:20:07 - mmengine - INFO - load 1297 Van database infos in DataBaseSampler
03/19 13:20:07 - mmengine - INFO - load 488 Truck database infos in DataBaseSampler
03/19 13:20:07 - mmengine - INFO - load 224 Tram database infos in DataBaseSampler
03/19 13:20:07 - mmengine - INFO - load 337 Misc database infos in DataBaseSampler
03/19 13:20:07 - mmengine - INFO - load 56 Person_sitting database infos in DataBaseSampler
03/19 13:20:07 - mmengine - INFO - After filter database:
03/19 13:20:07 - mmengine - INFO - load 2019 Pedestrian database infos in DataBaseSampler
03/19 13:20:07 - mmengine - INFO - load 10520 Car database infos in DataBaseSampler
03/19 13:20:07 - mmengine - INFO - load 564 Cyclist database infos in DataBaseSampl

/media/helloalone/新加卷/ubuntu_code/My_CMT/projects/mmdet3d_plugin/models/utils/grid_mask.py:114: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400400184/work/torch/csrc/utils/tensor_numpy.cpp:206.)
  mask = torch.from_numpy(mask).float().cuda()


In [5]:
img_feats[0].shape, img_feats[1].shape

(torch.Size([4, 256, 20, 60]), torch.Size([4, 256, 10, 30]))

In [6]:
voxel_features.shape, x1.shape, x2[0].shape, x2[1].shape, x3[0].shape

(torch.Size([48604, 4]),
 torch.Size([4, 256, 100, 100]),
 torch.Size([4, 128, 100, 100]),
 torch.Size([4, 256, 50, 50]),
 torch.Size([4, 512, 100, 100]))

In [7]:
# 训练过程
for data_batch in runner.train_dataloader:
    data_batch = runner.model.data_preprocessor(data_batch, training=True)
    if isinstance(data_batch, dict):
        losses = runner.model(**data_batch, mode='loss')
    elif isinstance(data_batch, (list, tuple)):
        losses = runner.model(*data_batch, mode='loss')
    else:
        raise TypeError()
    break

/media/helloalone/新加卷/ubuntu_anaconda_envs/python38/lib/python3.8/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/media/helloalone/新加卷/ubuntu_anaconda_envs/python38/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400400184/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:
losses

{'loss_cls': tensor(4.5400, device='cuda:0', grad_fn=<AddBackward0>),
 'loss_bbox': tensor(4.8334, device='cuda:0', grad_fn=<AddBackward0>),
 'd0.loss_cls': tensor(13.8242, device='cuda:0', grad_fn=<AddBackward0>),
 'd0.loss_bbox': tensor(4.6958, device='cuda:0', grad_fn=<AddBackward0>),
 'd1.loss_cls': tensor(7.4776, device='cuda:0', grad_fn=<AddBackward0>),
 'd1.loss_bbox': tensor(4.7611, device='cuda:0', grad_fn=<AddBackward0>),
 'd2.loss_cls': tensor(7.6146, device='cuda:0', grad_fn=<AddBackward0>),
 'd2.loss_bbox': tensor(4.0996, device='cuda:0', grad_fn=<AddBackward0>),
 'd3.loss_cls': tensor(6.0241, device='cuda:0', grad_fn=<AddBackward0>),
 'd3.loss_bbox': tensor(5.0156, device='cuda:0', grad_fn=<AddBackward0>),
 'd4.loss_cls': tensor(10.3131, device='cuda:0', grad_fn=<AddBackward0>),
 'd4.loss_bbox': tensor(4.7757, device='cuda:0', grad_fn=<AddBackward0>),
 'dn_loss_cls': tensor(2.3941, device='cuda:0', grad_fn=<AddBackward0>),
 'dn_loss_bbox': tensor(5.8194, device='cuda:0', 

In [ ]:
data_batch['inputs'], data_batch['data_samples'][0]

In [9]:
runner.model.eval()

CmtDetector(
  (data_preprocessor): Det3DDataPreprocessor()
  (pts_voxel_encoder): HardSimpleVFE()
  (pts_middle_encoder): SparseEncoder(
    (conv_input): SparseSequential(
      (0): SubMConv3d(4, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
      (1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (encoder_layers): SparseSequential(
      (encoder_layer1): SparseSequential(
        (0): SparseBasicBlock(
          (conv1): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
          (bn1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (conv2): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], ou

In [10]:
# 验证过程
for data_batch in runner.val_dataloader:
    data_batch = runner.model.data_preprocessor(data_batch, training=False)
    if isinstance(data_batch, dict):
        outputs = runner.model(**data_batch, mode='predict')
    elif isinstance(data_batch, (list, tuple)):
        outputs = runner.model(**data_batch, mode='predict')
    else:
        raise TypeError()
    runner.val_evaluator.process(data_samples=outputs, data_batch=data_batch)
    break
# with torch.no_grad():
#     metrics = runner.val_evaluator.evaluate(len(runner.val_dataloader.dataset))

03/19 13:20:36 - mmengine - INFO - ------------------------------
03/19 13:20:36 - mmengine - INFO - The length of test dataset: 3769
03/19 13:20:36 - mmengine - INFO - The number of instances per category in the dataset:
+------------+--------+
| category   | number |
+------------+--------+
| Pedestrian | 2280   |
| Cyclist    | 893    |
| Car        | 14385  |
+------------+--------+


/media/helloalone/新加卷/ubuntu_code/mmdetection3d/mmdet3d/evaluation/functional/kitti_utils/eval.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_thresholds(scores: np.ndarray, num_gt, num_sample_pts=41):


In [11]:
outputs[0]

<Det3DDataSample(

    META INFORMATION
    pcd_vertical_flip: False
    sample_idx: 0
    num_views: 1
    transformation_3d_flow: ['R', 'S', 'T']
    pcd_rotation_angle: 0.0
    lidar_path: 'data/kitti/training/velodyne_reduced/000001.bin'
    num_pts_feats: 4
    box_type_3d: <class 'mmdet3d.structures.bbox_3d.lidar_box3d.LiDARInstance3DBoxes'>
    pcd_rotation: tensor([[1., 0., 0.],
                [-0., 1., 0.],
                [0., 0., 1.]])
    box_mode_3d: <Box3DMode.LIDAR: 0>
    pad_shape: (320, 960)
    flip: False
    cam2img: array([[[6.24032621e+02, 0.00000000e+00, 4.78186434e+02, 4.48572800e+01],
                [0.00000000e+00, 6.24032621e+02, 1.49495355e+02, 2.16379100e-01],
                [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 2.74588400e-03],
                [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]])
    lidar2cam: array([[[ 2.34773921e-04, -9.99944150e-01, -1.05634769e-02,
                 -2.79681687e-03],
                [ 1.044

In [ ]:
runner.val_dataloader

In [13]:
time_start = time.time()
num = 0
with torch.no_grad():
    for data_batch in runner.val_dataloader:
        data_batch = runner.model.data_preprocessor(data_batch, training=False)
        if isinstance(data_batch, dict):
            outputs = runner.model(**data_batch, mode='predict')
        elif isinstance(data_batch, (list, tuple)):
            outputs = runner.model(**data_batch, mode='predict')
        else:
            raise TypeError()
        num += 1
        if num == 100:
            break
print(time.time() - time_start)

6.9588234424591064


In [20]:
time_start = time.time()
with torch.no_grad():
    data_batch_raw = next(iter(runner.val_dataloader))
    for _ in range(100):
        data_batch = runner.model.data_preprocessor(data_batch_raw, training=False)
        if isinstance(data_batch, dict):
            outputs = runner.model(**data_batch, mode='predict')
        elif isinstance(data_batch, (list, tuple)):
            outputs = runner.model(**data_batch, mode='predict')
        else:
            raise TypeError()
print(time.time() - time_start)

6.595309019088745


In [19]:
with torch.no_grad():
    data_batch = next(iter(runner.val_dataloader))
    data_batch = runner.model.data_preprocessor(data_batch, training=False)
    batch_inputs_dict = data_batch['inputs']
    batch_data_samples = data_batch['data_samples']
    imgs = batch_inputs_dict.get('imgs', None)
    points = batch_inputs_dict.get('points', None)
    img_metas = [item.metainfo for item in batch_data_samples]

    time_start = time.time()
    for _ in range(100):
        img_feats = runner.model.extract_img_feat(imgs, img_metas)
    
    midedle1 = time.time()
    for _ in range(100):
        pts_feats = runner.model.extract_pts_feat(points, img_feats, img_metas)

    midedle2 = time.time()
    for _ in range(100):
        outs = runner.model.pts_bbox_head(pts_feats, img_feats, img_metas)

    middle3 = time.time()
    for _ in range(100):
        bbox_list = runner.model.pts_bbox_head.get_bboxes(
            outs, img_metas, rescale=False)
        
        # bbox_results = []
        # for bboxes, scores, labels in bbox_list:
        #     results = InstanceData()
        #     results.bboxes_3d = bboxes.to('cpu')
        #     results.scores_3d = scores.cpu()
        #     results.labels_3d = labels.cpu()
        #     bbox_results.append(results)
        # detsamples = runner.model.add_pred_to_datasample(batch_data_samples,
        #                                             data_instances_3d = bbox_results,
        #                                             data_instances_2d = None)
    end = time.time()
    print(midedle1 - time_start, midedle2 - midedle1, middle3 - midedle2, end - middle3)

1.129737138748169 2.4780116081237793 2.382964849472046 0.0486757755279541
